In [2]:
import numpy as np
import pandas as pd
import scanpy as sc
import sys
sys.path.append("../utils/")
import pre_processing
import sci_excerpts
import anndata2ri
import rpy2.robjects as ro
import rpy2.rinterface_lib.callbacks

In [14]:
adata = sc.read_h5ad("../../process/pre-intergration/big_data/20241018_merge_all_step0.h5ad")

In [16]:
adata.X[1:10,1:10].toarray()

array([[0., 1., 0., 0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [2., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 2., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 1., 0., 0.]])

In [17]:
adata.obs.drop(['id', 'batch', 'ident'], axis=1, inplace=True)

In [18]:
adata.obs.drop('RNA_snn_res.0.5', axis=1, inplace=True)

In [19]:
adata.obs.drop(['percent_mito', 'integrated_snn_res.0.5', 'seurat_clusters'], axis=1, inplace=True)

In [20]:
for cat in adata.obs.columns:
    if adata.obs[cat].isnull().any():
        print(cat, "has null values")
    elif "nan" in adata.obs[cat]:
        print(cat, "has 'nan' values")

coarse_anno_1 has null values


In [21]:
adata.var["gene_symbols"] = adata.var_names

In [22]:
adata = pre_processing.add_cell_annotations(adata, var_index="gene_symbols")

In [23]:
n_genes_pre = adata.shape[1]
sc.pp.filter_genes(adata, min_cells=10)
n_genes_post = adata.shape[1]
print("Number of genes removed: " + str(n_genes_pre - n_genes_post))
print("Number of genes pre-filtering: " + str(n_genes_pre))
print("Number of genes post filtering: " + str(n_genes_post))

Number of genes removed: 36519
Number of genes pre-filtering: 72822
Number of genes post filtering: 36303


In [30]:
adata.obs[adata.obs["orig.ident"]=="Atlas_Jan_incisor_SS2"]

,orig.ident,nCount_RNA,nFeature_RNA,coarse_anno_1,total_counts,log10_total_counts,n_genes_detected,mito_frac,ribo_frac,compl
Atlas_Jan_Mouse_renamed_13_SS2_16_015_A1,Atlas_Jan_incisor_SS2,83471.0,2676,NaN,83471.0,4.921536,2676,0.0,0.006637,0.032059
Atlas_Jan_Mouse_renamed_13_SS2_16_015_A10,Atlas_Jan_incisor_SS2,236537.0,4810,NaN,236537.0,5.373899,4810,0.0,0.019561,0.020335
Atlas_Jan_Mouse_renamed_13_SS2_16_015_A11,Atlas_Jan_incisor_SS2,265886.0,3560,NaN,265886.0,5.424695,3560,0.0,0.006796,0.013389
Atlas_Jan_Mouse_renamed_13_SS2_16_015_A12,Atlas_Jan_incisor_SS2,210957.0,3115,NaN,210957.0,5.324194,3115,0.0,0.005845,0.014766
Atlas_Jan_Mouse_renamed_13_SS2_16_015_A13,Atlas_Jan_incisor_SS2,233626.0,4037,NaN,233626.0,5.368521,4037,0.0,0.007773,0.017280
...,...,...,...,...,...,...,...,...,...,...
Atlas_Jan_Mouse_renamed_13_SS2_17_014_P5,Atlas_Jan_incisor_SS2,380551.0,3255,NaN,380551.0,5.580413,3255,0.0,0.006501,0.008553
Atlas_Jan_Mouse_renamed_13_SS2_17_014_P6,Atlas_Jan_incisor_SS2,344760.0,5550,NaN,344760.0,5.537517,5550,0.0,0.016600,0.016098
Atlas_Jan_Mouse_renamed_13_SS2_17_014_P7,Atlas_Jan_incisor_SS2,386465.0,3688,NaN,386465.0,5.587110,3688,0.0,0.012159,0.009543
Atlas_Jan_Mouse_renamed_13_SS2_17_014_P8,Atlas_Jan_incisor_SS2,330018.0,4683,NaN,330018.0,5.518538,4683,0.0,0.008136,0.014190


In [26]:
adata.obs["coarse_anno_1"][adata.obs[""]]

ToothNc_Junjun_AAACCCAAGCCTGTCG-1_1       Epithelium
ToothNc_Junjun_AAACCCAAGCTAAGTA-1_1       Mesenchyme
ToothNc_Junjun_AAACCCAAGTGATTCC-1_1       Epithelium
ToothNc_Junjun_AAACCCACAAGATTGA-1_1              RBC
ToothNc_Junjun_AAACCCACACAACGAG-1_1      Endothelium
                                            ...     
MolarP12_Tomoko_TTTGGTTGTTGTCAGT-1_2    Perivascular
MolarP12_Tomoko_TTTGGTTTCCGTTTCG-1_2    Perivascular
MolarP12_Tomoko_TTTGGTTTCTAACACG-1_2          Neuron
MolarP12_Tomoko_TTTGTTGAGAGGTATT-1_2          Neuron
MolarP12_Tomoko_TTTGTTGTCTTACCGC-1_2          Immune
Name: coarse_anno_1, Length: 371467, dtype: category
Categories (8, object): ['Endothelium', 'Epithelium', 'Immune', 'Mesenchyme', 'Muscle', 'Neuron', 'Perivascular', 'RBC']

In [31]:
adata.write_h5ad("../../process/pre-intergration/big_data/20241019_mergeall_filter_gene_step1.h5ad")